In [ ]:
# 【1】 python调用js的库之execjs 安装及简介+ node.js：
'''
pip install PyExecJS

一.通过相关库调用JavaScript引擎执行JS
    PyExecJS，推荐，有多个引擎可选，一般选择NodeJS
    Node.js ,下载官网：https://nodejs.org/dist/v10.15.3/
    JScript :windows自带JS解释器，IE浏览器.在windows上不需要其他的依赖便可运行execjs，也可以调用其他的JS环境

    源码中给出， 可执行execjs的环境：
        PyV8         = "PyV8"
        Node.js         = "Node"
        JavaScriptCore    = "JavaScriptCore"
        SpiderMonkey    = "SpiderMonkey"
        JScript       = "JScript"
        PhantomJS      = "PhantomJS"
        SlimerJS       = "SlimerJS"
        Nashorn       = "Nashorn"
_____________________导入本地js文件___________________________________
    有时候js代码过长，我们可以将js代码先保存到文件中。
　　由于历史遗留问题，ExecJS以前使用python2编写的，所以在代码实现过程中会涉及到文件编码的问题。
  ExecJS先将js代码读到内存中，然后再把调用js的代码和js文件的代码一同写入到一个临时文件
  （C:\Users\user\AppData\Local\Temp\xx.js）中，如果js文件采用的是UTF-8编码，那么在写入到临时文件时，
  模块会报UnicodeEncodeError: 'gbk' codec can't encode character xxx ， 主要是因为模块在进行文件写入时
  采用的是windows的默认编码gbk，而没有指定encoding=utf-8，所以js文件需要以gbk编码保存。
_____________________ 通过os.environ 环境切换___________________________________
    os.environ["EXECJS_RUNTIME"] = "Node"
    execjs.get().name
    execjs.eval("1 + 2")
_____________________ 通过execjs.get  环境切换___________________________________
通过execjs.get 切换
jscript = execjs.get(execjs.runtime_names.JScript)  # runtime_names 便是execjs源码中给出的执行环境的。 
execjs.runtime_names.xxx  xxx必须在上一节 #3中取
jscript.eval("1 + 2")
——————————————————————————————————————————————————
（一）安装 Node.js 
https://nodejs.org/dist/v10.15.3/
'''

In [1]:
# 【1】 execjs + node.js : 检测运行环境 + 手动指定引擎
import execjs
import os
# ——————————————————————
print(execjs.get().name)   # 检测运行环境
default = execjs.get()
print(default.eval("1 + 2"))
print(execjs.get().eval('1+1'))
# —————————————————————— 手动指定引擎
# os.environ["EXECJS_RUNTIME"] = "Node"
# print(execjs.get().name)
# os.environ["EXECJS_RUNTIME"] = "JScript"
# print(execjs.get().name)
# —————————————————————— 手动指定引擎
# js1=execjs.get(execjs.runtime_names.JScript)
# print(js1.eval('1+2'))
# js2=execjs.get(execjs.runtime_names.Node)
# print(js2.eval('2+3'))


Node.js (V8)
3
2


In [ ]:
# （一）execjs 基础练习：
import execjs
'''____________________ execjs会自动使用当前电脑上的运行时环境,运行js：_______________________'''
# print(execjs.get().name) # 检测运行环境
# default = execjs.get()
# default.eval("1 + 2")
'''____________________ eval + split 直接运算_______________________'''
# print(execjs.eval("'red yellow blue'.split(' ')"))
'''____________________ compile + function + call / eval 编译后再调用 ___________________'''
# ctx = execjs.compile('''function add(x, y) 
#                         { return x + y};''')
# print(ctx.call("add", 1, 2))
# 使用eval的写法同上，但是在传入字符串或者其他类型的数据时需要添加对应的格式,如下所示，具体可在程序中debug
# print(ctx.eval("add({0}, {1})".format(1,2)))  
'''____________________ 调用本地 js 文件 + compile + eval ___________________'''
with open(r"C:\Users\hp\Desktop\python_js.txt", 'r') as f:
       content = f.read()  #读取js文件的全部内容到content变量中
ctx = execjs.compile(content)
jscode ="add({0}, {1})".format(1,2)
print(ctx.eval(jscode))


In [ ]:
# （一）execjs 基础练习：华氏度转换为摄氏度
import execjs
'''____________________ function 编译后再运算 并返回结果_________________________'''
# jsFormula='''function toCelsius(f) {
#     return (5/9) * (f-32);
# } '''
# jsFunc=execjs.compile(jsFormula)
# jsFunc.call('toCelsius',80)
# jsFunc.eval('toCelsius(80)')

'''____________________ split 分割对比 直接运算 与 编译后运算_________________________'''
# jsFomula="'Hello Girl'.split(' ')"
# execjs.eval(jsFomula)
# execjs.eval("'Hello Girl'.split(' ')")

jsFormula="function js_split(str_para){return str_para.split(' ');}"
jsFunc=execjs.compile(jsFormula)
print(jsFunc.call('js_split','Hello Girl'))
print(jsFunc.eval('js_split("Hello Girl")'))
'''____________________ var 定义变量 与 typeof _________________________'''
# jsFormula='function myfunc(){ var carName = "Volvo";return typeof carName + " " + carName;}'
# jsFunc=execjs.compile(jsFormula)
# jsFunc.eval('myfunc()')
'''____________________ var 定义对象object _________________________'''
# jsFormula='''function myfunc() {var person = {
#   firstName: "Bill",
#   lastName : "Gates",
#   id       : 678,
#   fullName : function() {return this.firstName + " " + this.lastName;}};
#   return person['firstName'] + "," + person.lastName + "," +person.fullName()+ "," +person.fullName;
# }''' 
# # this 指的是“拥有” fullName 函数的 person 对象
# jsFunc=execjs.compile(jsFormula)
# # jsFunc.eval('myfunc()')
# jsFunc.call('myfunc')

In [ ]:
# （二）百度翻译(request直接访问https://fanyi.baidu.com/?aldtype=16047#en/zh/失败)
import requests
from lxml import etree
class Bdfanyi():
    def __init__(self):
        self.url='https://fanyi.baidu.com/?aldtype=16047#en/zh/{}'
        self.header={
            'accept-encoding':'gzip, deflate, br',
            'accept-language': 'zh-CN,zh;q=0.9',
            'upgrade-insecure-requests': '1',
            'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36'
        }
    def get_Bdfy(self,url):
        res=requests.get(url=url,headers=self.header)
        html=res.content.decode(res.encoding)
        parse_html=etree.HTML(html)
        result=parse_html.xpath("//p[@class='ordinary-output target-output clearfix']//span/text()")
        print(result)
        
    def main(self):
        url=self.url.format('Hello')
        self.get_Bdfy(url)

if __name__=='__main__':
    start_bdfy=Bdfanyi()
    start_bdfy.main()

In [ ]:
# （二）百度翻译 sign参数 js解密：
import execjs
inputDate='hello'
with open(r'C:\Users\hp\Desktop\bdfy.txt','r') as f:
    jsFunction=f.read()
sign=execjs.compile(jsFunction).call('e',inputDate)
print(sign)
    

In [ ]:
# （二）百度翻译:
import requests
import re
import execjs
import json

with open(r'C:\Users\hp\Desktop\bdfy.txt','r') as f:
    jsFunction=f.read()
jsFunc=execjs.compile(jsFunction)

class BDfy():
    def __init__(self):
        self.tokenUrl=r'https://fanyi.baidu.com/?aldtype=16047#en/zh/{}'
        self.tokenHeader={
            'accept-encoding':'gzip, deflate, br',
            'accept-language': 'zh-CN,zh;q=0.9',
            'referer':'https://fanyi.baidu.com/?aldtype=16047',
            'upgrade-insecure-requests': '1',
            'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36',
        }
        self.transUrl='https://fanyi.baidu.com/v2transapi'
        
    def get_token(self,url):  # 获取 postdata 参数 token 
        res=requests.get(url=self.tokenUrl,headers=self.tokenHeader)
#         print(res.cookies)
        html=res.content.decode(res.encoding)
        result=re.compile("token: '(.{32})",re.S).search(html)
        token=result.groups()[0]
        return token
    def get_trans(self,inputData,token,sign):
        data={
            'from': 'en',
            'to': 'zh',
            'query': inputData,
            'transtype': 'enter',
            'simple_means_flag': '3',
            'sign': sign,
            'token':token,
            'domain': 'common',
        }
        print(data)
        res=requests.post(url=self.transUrl,headers=self.tokenHeader,data=data)
        jsonResult=json.loads(res.text)
        print(res.text)
    def main(self):
        inputData=input('请输入需要百度翻译的内容：')
        url=self.tokenUrl.format(inputData)
        token=self.get_token(url)    # 获取 postdata 参数 token 
        sign=jsFunc.call('e',inputData)
        print('token=',token)
        print('sign=',sign)
        self.get_trans(inputData,token,sign)

if __name__=='__main__':
    bdfy=BDfy()
    bdfy.main()

In [ ]:
# 【2】Js2Py 库： 不需要依赖别的环境，可以独立运行Js文件，运行速度会稍微慢
#  pip install Js2Py
import execjs
import js2py
import re
# ——————————————————————————————  常规操作
# print(js2py.eval_js('var a="hello";console.log(a)'))
# —————————————————————————————— 循环遍历
aa=js2py.eval_js("""
    for(var i=1;i<6;i++){
        console.log(i);
    }
""")
print(aa)

# —————————————————————————————— 读取Js文件
# 我们可以把JS文件写入文件中，以便我们进行调用，如下：
# function f(aa){
#     if(aa>11){
#         console.log('OK')

#     }else{
#         console.log('Fail')
#   }
# }
# ———————————
# with open(r'C:\Users\hp\Desktop\python_js.js','r') as f:
#     aa=js2py.eval_js(f.read())
#     print(aa(12))
